# Returns engine demo

Demonstrates usage of the returns engine, which consumes a backend db with "time", "symbol" and fair expressions

In [1]:
import mnemosyne as ms
from mnemosyne.engines import ReturnsEngine 
from datetime import datetime as Datetime
import polars as pl
from pathlib import Path
from tqdm.auto import tqdm 
from datetime import date as Date

# Minimal example

In [2]:
symbol_enum = pl.Enum(['BTC', 'ETH', 'BTCDOWN', 'BNB', 'SHIB'])

query_lf = pl.DataFrame({
    'symbol': ['BTC', 'ETH', 'BTCDOWN', 'BNB', 'SHIB'], 
    'times': [Datetime(2025, 8, 1, 9, 0, 0)] * 5
}).with_columns(pl.col('symbol').cast(symbol_enum)).lazy()

backend_db = ms.binance.BinanceLastTradesGrid(
            peg_symbol='USDT', 
            grid_interval='10m', 
            dataset_type=ms.DatasetType.BinanceSpotTrades, 
        ).lazyframe()

re = ReturnsEngine(
    backend_db, 
    backend_fair_expr = pl.col('vwap_total_by_base'), 
    backend_time_expr = pl.col('last_trade_time')
)

value = re.query(query_lf, 
    start_time_expr = pl.col('times'), 
    mark_duration=pl.lit('10m'), 
    tick_lag_tolerance=pl.lit('30s'), 
    verbose_debug=False
)
value.collect()

/home/nlyu/Code/midas/mnemosyne/python/mnemosyne/engines/returns.py:71: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  for col in result.columns if col.endswith('_0')
sys:1: UserWarning: Sortedness of columns cannot be checked when 'by' groups provided


symbol,times,max_tick_to_query_lag,return
enum,datetime[μs],duration[μs],f64
"""BTC""",2025-08-01 09:00:00,154216µs,0.002026
"""ETH""",2025-08-01 09:00:00,677388µs,0.001934
"""BTCDOWN""",2025-08-01 09:00:00,null,null
"""BNB""",2025-08-01 09:00:00,1s 191853µs,0.001734
"""SHIB""",2025-08-01 09:00:00,6s 436199µs,0.002999


In [3]:
import hvplot.polars
import hvplot 
hvplot.extension('plotly')


save_root = Path('~/Documents/binance_gridded_returns')

pivot_dates = [
    Date(2022, 1, 1),
    Date(2022, 9, 1),
    Date(2023, 6, 1),
    Date(2024, 3, 1),
    Date(2025, 1, 1), 
    Date(2026, 1, 1)
]

mark_exprs = {
    'now_to_p10m': (pl.col('time'), pl.lit('10m')),
    'p1m_to_p11m': (pl.col('time').dt.offset_by('1m'), pl.lit('10m')),
    'm10m_to_now': (pl.col('time').dt.offset_by('-10m'), pl.lit('10m')),
    'm20m_to_now': (pl.col('time').dt.offset_by('-20m'), pl.lit('20m')),
    'm30m_to_now': (pl.col('time').dt.offset_by('-30m'), pl.lit('30m')),
}

# Collect 10m-gridded data

In [4]:
query_lf = ms.binance.BinanceLastTradesGrid(
    peg_symbol='USDT',
    grid_interval='10m',
    dataset_type=ms.DatasetType.BinanceSpotTrades, 
).lazyframe()

backend_db = ms.binance.BinanceLastTradesGrid(
            peg_symbol='USDT', 
            grid_interval='4s', 
            dataset_type=ms.DatasetType.BinanceSpotTrades, 
        ).lazyframe()

re = ReturnsEngine(
    backend_db, 
    backend_fair_expr = pl.col('vwap_total_by_base'), 
    backend_time_expr = pl.col('last_trade_time')
)

In [ ]:
for j in tqdm(range(len(pivot_dates) - 1)):
    query_slice = query_lf.filter(pl.col('date').is_between(pivot_dates[j], pivot_dates[j+1], closed='left'))
    value_lf = re.query_batch(
        query_slice,
        mark_exprs=mark_exprs,
        tick_lag_tolerance=pl.lit('10m'),
        append_lag=False)
    value_lf.sink_parquet(save_root / f'binance_spot_10m_grid_mark10m_{j}.parquet', compression='brotli')

  0%|          | 0/5 [00:00<?, ?it/s]

In [6]:
universe_df = pl.scan_parquet(save_root / 'binance_spot_10m_grid_mark10m_*.parquet').collect()
print(universe_df.shape)
universe_df.group_by('date').agg(pl.col('symbol').unique().len()).sort('date').plot.line(x='date', y='symbol')

(73166941, 22)


alt.Chart(...)

# Perp data

In [7]:
query_lf = ms.binance.BinanceLastTradesGrid(
    peg_symbol='USDT',
    grid_interval='10m',
    dataset_type=ms.DatasetType.BinanceUmPerpTrades, 
).lazyframe()

backend_db = ms.binance.BinanceLastTradesGrid(
            peg_symbol='USDT', 
            grid_interval='4s', 
            dataset_type=ms.DatasetType.BinanceUmPerpTrades, 
        ).lazyframe()

re = ReturnsEngine(
    backend_db, 
    backend_fair_expr = pl.col('vwap_total_by_base'), 
    backend_time_expr = pl.col('last_trade_time')
)

In [8]:
for j in tqdm(range(len(pivot_dates) - 1)):
    print(pivot_dates[j], pivot_dates[j+1])
    query_slice = query_lf.filter(pl.col('date').is_between(pivot_dates[j], pivot_dates[j+1], closed='left'))
    value_lf = re.query_batch(
        query_slice,
        mark_exprs=mark_exprs,
        tick_lag_tolerance=pl.lit('10m'),
        append_lag=False)
    value_lf.sink_parquet(save_root / f'binance_futures_10m_grid_mark10m_{j}.parquet', compression='brotli')

  0%|          | 0/5 [00:00<?, ?it/s]

2022-01-01 2022-09-01


/tmp/ipykernel_2307826/2426307499.py:9: UserWarning:

Sortedness of columns cannot be checked when 'by' groups provided



2022-09-01 2023-06-01
2023-06-01 2024-03-01
2024-03-01 2025-01-01
2025-01-01 2026-01-01


In [9]:
universe_df = pl.scan_parquet(save_root / 'binance_futures_10m_grid_mark10m_*.parquet').collect()
print(universe_df.shape)
universe_df.group_by('date').agg(pl.col('symbol').unique().len()).sort('date').plot.line(x='date', y='symbol')

(50030762, 22)


alt.Chart(...)

In [10]:
universe_df

symbol,date,time,peg_symbol,open,high,low,close,volume_base,volume_quote,trade_count,last_trade_time,taker_buy_volume_quote,taker_sell_volume_quote,vwap_taker_buy,vwap_taker_sell,vwap_total_by_base,return_now_to_p10m,return_p1m_to_p11m,return_m10m_to_now,return_m20m_to_now,return_m30m_to_now
enum,date,datetime[μs],str,f64,f64,f64,f64,f64,f64,u32,datetime[μs],f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""FTM""",2022-01-01,2022-01-01 00:00:00,"""USDT""",2.2498,2.2826,2.2486,2.2644,2.934305e6,6.6539e6,11749,2022-01-01 00:09:59.610,3.8383e6,2.8156e6,2.267892,2.267324,2.267625,null,0.003267,null,null,null
"""ONE""",2022-01-01,2022-01-01 00:00:00,"""USDT""",0.23349,0.23606,0.23341,0.23433,5.184906e6,1.2168e6,2767,2022-01-01 00:09:56.880,709967.39566,506863.32764,0.234743,0.234612,0.234687,null,0.003559,null,null,null
"""AVAX""",2022-01-01,2022-01-01 00:00:00,"""USDT""",109.454,110.581,109.428,110.06,60196.0,6.6233e6,7105,2022-01-01 00:09:59.992,3.7942e6,2.8291e6,110.041296,110.012547,110.028467,null,0.004354,null,null,null
"""EGLD""",2022-01-01,2022-01-01 00:00:00,"""USDT""",238.65,240.49,238.62,240.24,1492.6,357905.439,1135,2022-01-01 00:09:59.901,191640.002,166265.437,239.760747,239.818087,239.786573,null,0.005392,null,null,null
"""RAY""",2022-01-01,2022-01-01 00:00:00,"""USDT""",6.339,6.378,6.339,6.369,15555.7,98939.003,518,2022-01-01 00:09:59.961,55429.503,43509.5,6.361363,6.359018,6.360317,null,0.00378,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""OM""",2025-03-14,2025-03-14 16:50:00,"""USDT""",6.26833,6.30198,6.26657,6.30196,24888.4,156398.906821,2293,2025-03-14 16:59:59.371,89592.025413,66806.881408,6.287222,6.279744,6.284008,0.005252,0.005269,-0.002582,-0.000304,0.001571
"""APE""",2025-01-11,2025-01-11 17:30:00,"""USDT""",1.1053,1.1071,1.1034,1.1049,352302.0,389283.8202,2055,2025-01-11 17:39:56.129,211075.7119,178208.1083,1.10519,1.104715,1.104972,-0.000113,-0.000115,0.001002,-0.004212,-0.006075
"""ANKR""",2025-02-20,2025-02-20 20:50:00,"""USDT""",0.02404,0.02408,0.02403,0.02407,1.386725e6,33366.08778,409,2025-02-20 20:59:58.986,24258.6421,9107.44568,0.024065,0.024051,0.024061,0.001553,0.000286,0.002502,0.003339,0.000833


# Test cells: batch & separate query should be equivalent

In [13]:
# Use short names - 'return_' prefix will be added automatically by query_batch
appends = [
    ('0m', '10m', 'now_to_p10m'),
    ('1m', '10m', 'p1m_to_p11m'),
    ('-10m', '10m', 'm10m_to_now'),
    ('-20m', '20m', 'm20m_to_now'),
    ('-30m', '30m', 'm30m_to_now'),
]

# OLD get_returns_col function for single queries (kept for compatibility)
# Returns a dataframe with one column corresponding to the given returns
def get_returns_col(re, query_lf, start_offset, mark_offset, col_name, **kwargs) -> pl.LazyFrame:
    return re.query(query_lf,
            start_time_expr = pl.col('time').dt.offset_by(start_offset), 
            mark_duration=pl.lit(mark_offset), 
            tick_lag_tolerance=pl.lit('10m'), 
            # append_lag=False,
            append_lag=True, 
            append_query_tick_times=True, 
            append_start_end_fairs=True, 
            **kwargs
        ).select(pl.col('return').alias(f'return_{col_name}'), pl.col('^.*query_time$').name.suffix(f'_{col_name}'), pl.col('^.*_fair$').name.suffix(f'_{col_name}'))

In [14]:
query_lf = ms.binance.BinanceLastTradesGrid(
    peg_symbol='USDT',
    grid_interval='10m',
    dataset_type=ms.DatasetType.BinanceUmPerpTrades, 
).lazyframe()

backend_db = ms.binance.BinanceLastTradesGrid(
            peg_symbol='USDT', 
            grid_interval='4s', 
            dataset_type=ms.DatasetType.BinanceUmPerpTrades, 
        ).lazyframe()

re = ReturnsEngine(
    backend_db, 
    backend_fair_expr = pl.col('vwap_total_by_base'), 
    backend_time_expr = pl.col('last_trade_time')
)

In [15]:
# OLD APPROACH: Multiple separate queries
query_lf_slice = query_lf.filter(pl.col('date') >= Date(2025, 10, 20))

benchmark_df = pl.concat([
    query_lf_slice, 
    *[get_returns_col(re, query_lf_slice, *a) for a in appends]
], how='horizontal').sort('symbol', 'time').collect()
benchmark_df

sys:1: UserWarning:

Sortedness of columns cannot be checked when 'by' groups provided



symbol,date,time,peg_symbol,open,high,low,close,volume_base,volume_quote,trade_count,last_trade_time,taker_buy_volume_quote,taker_sell_volume_quote,vwap_taker_buy,vwap_taker_sell,vwap_total_by_base,return_now_to_p10m,start_query_time_now_to_p10m,end_query_time_now_to_p10m,start_fair_now_to_p10m,end_fair_now_to_p10m,return_p1m_to_p11m,start_query_time_p1m_to_p11m,end_query_time_p1m_to_p11m,start_fair_p1m_to_p11m,end_fair_p1m_to_p11m,return_m10m_to_now,start_query_time_m10m_to_now,end_query_time_m10m_to_now,start_fair_m10m_to_now,end_fair_m10m_to_now,return_m20m_to_now,start_query_time_m20m_to_now,end_query_time_m20m_to_now,start_fair_m20m_to_now,end_fair_m20m_to_now,return_m30m_to_now,start_query_time_m30m_to_now,end_query_time_m30m_to_now,start_fair_m30m_to_now,end_fair_m30m_to_now
enum,date,datetime[μs],str,f64,f64,f64,f64,f64,f64,u32,datetime[μs],f64,f64,f64,f64,f64,f64,datetime[μs],datetime[μs],f64,f64,f64,datetime[μs],datetime[μs],f64,f64,f64,datetime[μs],datetime[μs],f64,f64,f64,datetime[μs],datetime[μs],f64,f64,f64,datetime[μs],datetime[μs],f64,f64
"""0G""",2025-10-20,2025-10-20 00:00:00,"""USDT""",1.8076,1.8078,1.7933,1.7933,103284.0,185931.5632,2672,2025-10-20 00:09:59.164,74274.6784,111656.8848,1.800168,1.800229,1.800197,null,2025-10-20 00:00:00,2025-10-20 00:10:00,null,1.793785,-0.005582,2025-10-20 00:01:00,2025-10-20 00:11:00,1.80578,1.7957,-0.005244,2025-10-19 23:50:00,2025-10-20 00:00:00,1.817327,1.807796,-0.007796,2025-10-19 23:40:00,2025-10-20 00:00:00,1.822,1.807796,-0.013158,2025-10-19 23:30:00,2025-10-20 00:00:00,1.8319,1.807796
"""0G""",2025-10-20,2025-10-20 00:10:00,"""USDT""",1.7934,1.7964,1.7827,1.7836,106765.0,191085.2464,2330,2025-10-20 00:19:59.245,89936.1106,101149.1358,1.790027,1.78956,1.789774,-0.005634,2025-10-20 00:10:00,2025-10-20 00:20:00,1.793785,1.783679,-0.009149,2025-10-20 00:11:00,2025-10-20 00:21:00,1.7957,1.77927,-0.00775,2025-10-20 00:00:00,2025-10-20 00:10:00,1.807796,1.793785,-0.012954,2025-10-19 23:50:00,2025-10-20 00:10:00,1.817327,1.793785,-0.015486,2025-10-19 23:40:00,2025-10-20 00:10:00,1.822,1.793785
"""0G""",2025-10-20,2025-10-20 00:20:00,"""USDT""",1.7831,1.7842,1.7558,1.7615,283575.0,500800.0332,5186,2025-10-20 00:29:59.826,247364.5675,253435.4657,1.76568,1.766412,1.766023,-0.012666,2025-10-20 00:20:00,2025-10-20 00:30:00,1.783679,1.761086,-0.013026,2025-10-20 00:21:00,2025-10-20 00:31:00,1.77927,1.756094,-0.005634,2025-10-20 00:10:00,2025-10-20 00:20:00,1.793785,1.783679,-0.013341,2025-10-20 00:00:00,2025-10-20 00:20:00,1.807796,1.783679,-0.018515,2025-10-19 23:50:00,2025-10-20 00:20:00,1.817327,1.783679
"""0G""",2025-10-20,2025-10-20 00:30:00,"""USDT""",1.7606,1.7611,1.744,1.7496,284953.0,498927.6529,4977,2025-10-20 00:39:56.510,182136.6942,316790.9587,1.751117,1.750805,1.750912,-0.006763,2025-10-20 00:30:00,2025-10-20 00:40:00,1.761086,1.749176,-0.000781,2025-10-20 00:31:00,2025-10-20 00:41:00,1.756094,1.754723,-0.012666,2025-10-20 00:20:00,2025-10-20 00:30:00,1.783679,1.761086,-0.018229,2025-10-20 00:10:00,2025-10-20 00:30:00,1.793785,1.761086,-0.025838,2025-10-20 00:00:00,2025-10-20 00:30:00,1.807796,1.761086
"""0G""",2025-10-20,2025-10-20 00:40:00,"""USDT""",1.7495,1.7601,1.7495,1.7529,199545.0,350364.6561,4126,2025-10-20 00:49:57.535,193507.7382,156856.9179,1.755955,1.755654,1.755818,0.001942,2025-10-20 00:40:00,2025-10-20 00:50:00,1.749176,1.752572,-0.000223,2025-10-20 00:41:00,2025-10-20 00:51:00,1.754723,1.754331,-0.006763,2025-10-20 00:30:00,2025-10-20 00:40:00,1.761086,1.749176,-0.019344,2025-10-20 00:20:00,2025-10-20 00:40:00,1.783679,1.749176,-0.024869,2025-10-20 00:10:00,2025-10-20 00:40:00,1.793785,1.749176
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""币安人生""",2025-10-23,2025-10-23 23:10:00,"""USDT""",0.25584,0.25889,0.25328,0.25413,6.875649e6,1.7642e6,14085,2025-10-23 23:19:59.008,864618.91363,899609.79566,0.256755,0.256448,0.256591,-0.006381,2025-10-23 23:10:00,2025-10-23 23:20:00,0.2558,0.254168,-0.013546,2025-10-23 2

In [16]:
# NEW APPROACH: Single batch query - much faster!
query_lf_slice = query_lf.filter(pl.col('date') >= Date(2025, 10, 20))

# Convert appends to mark_exprs for query_batch
mark_exprs = {
    'now_to_p10m': (pl.col('time'), pl.lit('10m')),
    'p1m_to_p11m': (pl.col('time').dt.offset_by('1m'), pl.lit('10m')),
    'm10m_to_now': (pl.col('time').dt.offset_by('-10m'), pl.lit('10m')),
    'm20m_to_now': (pl.col('time').dt.offset_by('-20m'), pl.lit('20m')),
    'm30m_to_now': (pl.col('time').dt.offset_by('-30m'), pl.lit('30m')),
}

answer_df = re.query_batch(
    query_lf_slice,
    mark_exprs=mark_exprs,
    tick_lag_tolerance=pl.lit('10m'),
    # append_lag=False
    append_lag=True, 
    append_query_tick_times=True, 
    append_start_end_fairs=True
).sort('symbol', 'time').collect()

# Verify same column names as old approach
print(f"Column names: {answer_df.columns}")
answer_df

Column names: ['symbol', 'date', 'time', 'peg_symbol', 'open', 'high', 'low', 'close', 'volume_base', 'volume_quote', 'trade_count', 'last_trade_time', 'taker_buy_volume_quote', 'taker_sell_volume_quote', 'vwap_taker_buy', 'vwap_taker_sell', 'vwap_total_by_base', 'start_query_time_now_to_p10m', 'end_query_time_now_to_p10m', 'start_tick_time_now_to_p10m', 'end_tick_time_now_to_p10m', 'max_tick_to_query_lag_now_to_p10m', 'start_fair_now_to_p10m', 'end_fair_now_to_p10m', 'return_now_to_p10m', 'start_query_time_p1m_to_p11m', 'end_query_time_p1m_to_p11m', 'start_tick_time_p1m_to_p11m', 'end_tick_time_p1m_to_p11m', 'max_tick_to_query_lag_p1m_to_p11m', 'start_fair_p1m_to_p11m', 'end_fair_p1m_to_p11m', 'return_p1m_to_p11m', 'start_query_time_m10m_to_now', 'end_query_time_m10m_to_now', 'start_tick_time_m10m_to_now', 'end_tick_time_m10m_to_now', 'max_tick_to_query_lag_m10m_to_now', 'start_fair_m10m_to_now', 'end_fair_m10m_to_now', 'return_m10m_to_now', 'start_query_time_m20m_to_now', 'end_query_

symbol,date,time,peg_symbol,open,high,low,close,volume_base,volume_quote,trade_count,last_trade_time,taker_buy_volume_quote,taker_sell_volume_quote,vwap_taker_buy,vwap_taker_sell,vwap_total_by_base,start_query_time_now_to_p10m,end_query_time_now_to_p10m,start_tick_time_now_to_p10m,end_tick_time_now_to_p10m,max_tick_to_query_lag_now_to_p10m,start_fair_now_to_p10m,end_fair_now_to_p10m,return_now_to_p10m,start_query_time_p1m_to_p11m,end_query_time_p1m_to_p11m,start_tick_time_p1m_to_p11m,end_tick_time_p1m_to_p11m,max_tick_to_query_lag_p1m_to_p11m,start_fair_p1m_to_p11m,end_fair_p1m_to_p11m,return_p1m_to_p11m,start_query_time_m10m_to_now,end_query_time_m10m_to_now,start_tick_time_m10m_to_now,end_tick_time_m10m_to_now,max_tick_to_query_lag_m10m_to_now,start_fair_m10m_to_now,end_fair_m10m_to_now,return_m10m_to_now,start_query_time_m20m_to_now,end_query_time_m20m_to_now,start_tick_time_m20m_to_now,end_tick_time_m20m_to_now,max_tick_to_query_lag_m20m_to_now,start_fair_m20m_to_now,end_fair_m20m_to_now,return_m20m_to_now,start_query_time_m30m_to_now,end_query_time_m30m_to_now,start_tick_time_m30m_to_now,end_tick_time_m30m_to_now,max_tick_to_query_lag_m30m_to_now,start_fair_m30m_to_now,end_fair_m30m_to_now,return_m30m_to_now
enum,date,datetime[μs],str,f64,f64,f64,f64,f64,f64,u32,datetime[μs],f64,f64,f64,f64,f64,datetime[μs],datetime[μs],datetime[μs],datetime[μs],duration[μs],f64,f64,f64,datetime[μs],datetime[μs],datetime[μs],datetime[μs],duration[μs],f64,f64,f64,datetime[μs],datetime[μs],datetime[μs],datetime[μs],duration[μs],f64,f64,f64,datetime[μs],datetime[μs],datetime[μs],datetime[μs],duration[μs],f64,f64,f64,datetime[μs],datetime[μs],datetime[μs],datetime[μs],duration[μs],f64,f64,f64
"""0G""",2025-10-20,2025-10-20 00:00:00,"""USDT""",1.8076,1.8078,1.7933,1.7933,103284.0,185931.5632,2672,2025-10-20 00:09:59.164,74274.6784,111656.8848,1.800168,1.800229,1.800197,2025-10-20 00:00:00,2025-10-20 00:10:00,2025-10-19 23:59:59.472,2025-10-20 00:09:59.164,836ms,1.807796,1.793785,-0.00775,2025-10-20 00:01:00,2025-10-20 00:11:00,2025-10-20 00:00:58.526,2025-10-20 00:10:56.332,3s 668ms,1.80578,1.7957,-0.005582,2025-10-19 23:50:00,2025-10-20 00:00:00,2025-10-19 23:49:58.588,2025-10-19 23:59:59.472,1s 412ms,1.817327,1.807796,-0.005244,2025-10-19 23:40:00,2025-10-20 00:00:00,2025-10-19 23:39:57.831,2025-10-19 23:59:59.472,2s 169ms,1.822,1.807796,-0.007796,2025-10-19 23:30:00,2025-10-20 00:00:00,2025-10-19 23:29:52.543,2025-10-19 23:59:59.472,7s 457ms,1.8319,1.807796,-0.013158
"""0G""",2025-10-20,2025-10-20 00:10:00,"""USDT""",1.7934,1.7964,1.7827,1.7836,106765.0,191085.2464,2330,2025-10-20 00:19:59.245,89936.1106,101149.1358,1.790027,1.78956,1.789774,2025-10-20 00:10:00,2025-10-20 00:20:00,2025-10-20 00:09:59.164,2025-10-20 00:19:59.245,836ms,1.793785,1.783679,-0.005634,2025-10-20 00:11:00,2025-10-20 00:21:00,2025-10-20 00:10:56.332,2025-10-20 00:20:59.432,3s 668ms,1.7957,1.77927,-0.009149,2025-10-20 00:00:00,2025-10-20 00:10:00,2025-10-19 23:59:59.472,2025-10-20 00:09:59.164,836ms,1.807796,1.793785,-0.00775,2025-10-19 23:50:00,2025-10-20 00:10:00,2025-10-19 23:49:58.588,2025-10-20 00:09:59.164,1s 412ms,1.817327,1.793785,-0.012954,2025-10-19 23:40:00,2025-10-20 00:10:00,2025-10-19 23:39:57.831,2025-10-20 00:09:59.164,2s 169ms,1.822,1.793785,-0.015486
"""0G""",2025-10-20,2025-10-20 00:20:00,"""USDT""",1.7831,1.7842,1.7558,1.7615,283575.0,500800.0332,5186,2025-10-20 00:29:59.826,247364.5675,253435.4657,1.76568,1.766412,1.766023,2025-10-20 00:20:00,2025-10-20 00:30:00,2025-10-20 00:19:59.245,2025-10-20 00:29:59.826,755ms,1.783679,1.761086,-0.012666,2025-10-20 00:21:00,2025-10-20 00:31:00,2025-10-20 00:20:59.432,2025-10-20 00:30:59.537,568ms,1.77927,1.756094,-0.013026,2025-10-20 00:10:00,2025-10-20 00:20:00,2025-10-20 00:09:59.164,2025-10-20 00:19:59.245,836ms,1.793785,1.783679,-0.005634,2025-10-20 00:00:00,2025-10-20 00:20:00,2025-10-19 23:59:59.472,2025-10-20 00:19:59.245,755ms,1.807796,1.783679,-0.013341,2025-10-19 23:50:00,2025-10-20 00:20:00,2025-1

In [18]:
# Double-check: this should be all true if not null!!
(answer_df.select('^return_.*$') == benchmark_df.select('^return_.*$'))

return_now_to_p10m,return_p1m_to_p11m,return_m10m_to_now,return_m20m_to_now,return_m30m_to_now
bool,bool,bool,bool,bool
null,true,true,true,true
true,true,true,true,true
true,true,true,true,true
true,true,true,true,true
true,true,true,true,true
…,…,…,…,…
true,true,true,true,true
true,true,true,true,true
true,true,true,true,true


# Debugging script

Used during dev & debugging

In [ ]:
mark_exprs, append_cols, query_lf_withidx = re.query_batch(
    query_lf_slice,
    mark_exprs=mark_exprs,
    tick_lag_tolerance=pl.lit('10m'),
    # append_lag=False
    append_lag=True, 
    append_query_tick_times=True, 
    append_start_end_fairs=True
)

schema = append_cols.collect_schema()
metric_cols = [c for c in schema.names() if c not in ['row_id', 'return_col_name']]

wide_append_cols = None
for ret_col_name in mark_exprs.keys():
    filtered = (
        append_cols
        .filter(pl.col('return_col_name') == ret_col_name)
        .select(
            'row_id',
            *[pl.col(col).alias(f'{col}_{ret_col_name}') for col in metric_cols]
        )
    )

    if wide_append_cols is None:
        wide_append_cols = filtered
    else:
        wide_append_cols = wide_append_cols.join(filtered, on='row_id', how='left')



# Join back to original query
result = query_lf_withidx.join(
    wide_append_cols, on='row_id',
    how='left'
).drop('row_id')

answer_df = result.sort('symbol', 'time').collect()

append_cols.collect()